# Cleaning The Metropolitan Museum of Art Open Access dataset

The Metropolitan Museum of Art presents over 5,000 years of art from around the world.

The Metropolitan Museum of Art provides select datasets of information on more than 480,000 artworks in its Collection. This work is published from: The United States Of America. The datasets support the search, use, and interaction with the Museum’s collection.

At this time, the datasets are available in CSV format, encoded in UTF-8.

Issues: Missing values, inconsistent information, missing documentation, possible duplication, mixed text and numeric data.

## Libraries

In [169]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', matplotlib.__version__)

numpy version: 1.26.4
pandas version: 2.1.4
scikit-learn version: 1.2.2
matplotlib version: 3.8.0


## Add data source

In [170]:
df_met_objects = pd.read_csv('openaccess/MetObjects.csv', low_memory=False)

## Inspect dataset

In [171]:
df_met_objects.info()
df_met_objects.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484956 entries, 0 to 484955
Data columns (total 54 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Object Number            484956 non-null  object 
 1   Is Highlight             484956 non-null  bool   
 2   Is Timeline Work         484956 non-null  bool   
 3   Is Public Domain         484956 non-null  bool   
 4   Object ID                484956 non-null  int64  
 5   Gallery Number           49541 non-null   object 
 6   Department               484956 non-null  object 
 7   AccessionYear            481094 non-null  object 
 8   Object Name              482690 non-null  object 
 9   Title                    456153 non-null  object 
 10  Culture                  208190 non-null  object 
 11  Period                   91143 non-null   object 
 12  Dynasty                  23201 non-null   object 
 13  Reign                    11236 non-null   object 
 14  Port

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
count,484956,484956,484956,484956,484956.000000,49541,484956,481094,482690,456153,...,2092,406239,24529,484956,69154,0.0,484956,192455,192455,192455
unique,481656,2,2,2,NaN,379,19,180,28631,245800,...,228,1244,1507,484956,69076,NaN,1,44171,43699,43886
top,62.635,False,False,True,NaN,774,Drawings and Prints,1963,Print,Terracotta fragment of a kylix (drinking cup),...,Upper Sepik River,Prints,"© Walker Evans Archive, The Metropolitan Museu...",http://www.metmuseum.org/art/collection/search/1,https://www.wikidata.org/wiki/Q97732991,NaN,"Metropolitan Museum of Art, New York, NY",Flowers,http://vocab.getty.edu/page/aat/300132399,https://www.wikidata.org/wiki/Q506
freq,4,482179,476977,248472,NaN,7547,172630,41980,102986,6415,...,362,84326,7364,1,17,NaN,484956,8543,8543,8543
mean,NaN,NaN,NaN,NaN,394499.940353,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,242732.561637,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,212379.750000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,374771.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,570971.250000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
df_met_objects.drop_duplicates()

# Rename columns to lowercase and replace whitespace with underscores
df_met_objects.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
# accession_year was formatted as Pascal, so it needed manual separation
df_met_objects.rename(columns={'accessionyear': 'accession_year'}, inplace=True)

df_met_objects.drop(inplace=True, columns=['object_number', 'artist_ulan_url', 'artist_wikidata_url', 'artist_alpha_sort', 'object_date', 'rights_and_reproduction', 'link_resource', 'object_wikidata_url', 'metadata_date', 'repository', 'tags', 'tags_aat_url', 'tags_wikidata_url' ])

# TODO dtypes, Transform | character
# Suggestion gender decider based on name if the value is NaN

df_met_objects.accession_year = pd.to_datetime(df_met_objects.accession_year, format='mixed')

df_met_objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484956 entries, 0 to 484955
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   is_highlight         484956 non-null  bool          
 1   is_timeline_work     484956 non-null  bool          
 2   is_public_domain     484956 non-null  bool          
 3   object_id            484956 non-null  int64         
 4   gallery_number       49541 non-null   object        
 5   department           484956 non-null  string        
 6   accession_year       481094 non-null  datetime64[ns]
 7   object_name          482690 non-null  object        
 8   title                456153 non-null  object        
 9   culture              208190 non-null  object        
 10  period               91143 non-null   object        
 11  dynasty              23201 non-null   object        
 12  reign                11236 non-null   object        
 13  portfolio     

In [178]:
df_met_objects

,is_highlight,is_timeline_work,is_public_domain,object_id,gallery_number,department,accession_year,object_name,title,culture,...,state,county,country,region,subregion,locale,locus,excavation,river,classification
0,False,False,False,1,NaN,The American Wing,1979-01-01,Coin,One-dollar Liberty Head Coin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,False,2,NaN,The American Wing,1980-01-01,Coin,Ten-dollar Liberty Head Coin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,False,3,NaN,The American Wing,1967-01-01,Coin,Two-and-a-Half Dollar Coin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,False,4,NaN,The American Wing,1967-01-01,Coin,Two-and-a-Half Dollar Coin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,5,NaN,The American Wing,1967-01-01,Coin,Two-and-a-Half Dollar Coin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484951,False,False,False,900605,NaN,Drawings and Prints,1955-01-01,Print,Holiday Card,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints
484952,False,False,False,900606,NaN,Drawings and Prints,1977-01-01,Print,Brooklyn Local,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints
484953,False,False,False,900633,NaN,Drawings and Prints,1933-01-01,Print,Yesterday and Today,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints
484954,True,False,False,900717,NaN,The Libraries,NaT,NaN,"De la loi du contraste simultané des couleurs,...",NaN,...,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
# df_met_objects.drop(columns=[df_met_objects.Ar, 'Co'])
s = df_met_objects.locus.astype(pd.StringDtype())

In [176]:


df_met_objects.department = df_met_objects.department.astype(pd.StringDtype())

df_met_objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484956 entries, 0 to 484955
Data columns (total 54 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   object_number            484956 non-null  object        
 1   is_highlight             484956 non-null  bool          
 2   is_timeline_work         484956 non-null  bool          
 3   is_public_domain         484956 non-null  bool          
 4   object_id                484956 non-null  int64         
 5   gallery_number           49541 non-null   object        
 6   department               484956 non-null  string        
 7   accession_year           481094 non-null  datetime64[ns]
 8   object_name              482690 non-null  object        
 9   title                    456153 non-null  object        
 10  culture                  208190 non-null  object        
 11  period                   91143 non-null   object        
 12  dynasty         